# introduce
In this experiment, i try to get a nice score through some basic models

I'm not good at FE, so,i only add some new features, and for Imputer and Scaler, i just let them be done by autoML tool, and I think the different preprocessing methods are good for Ensemble/stacking.

In short,my method is like:
1. get original data
2. send to Hypergbm
3. get TopN best models
4. run GreedyEnsemble


About  PLB:


    a. best xgb model, **PLB:0.81793**
    
    b. best lgb model, **PLB:0.81767**
    
    c. best cat model, **PLB:0.81744**
    
    e. GreedyEnsemble, **PLB:0.81825**
    
There may also be performance improvements in basic models. Since we know trainning cost lot's of time in this competition.

Hope it can be helpful for u.


In [ ]:
# #install HyperGBM
!pip3 install -U hypergbm
!pip3 install -U scikit-learn==0.23.2

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings("ignore")

In [ ]:
##load data
train = pd.read_csv("/kaggle/input/tabular-playground-series-sep-2021/train.csv", index_col=0)
test = pd.read_csv("/kaggle/input/tabular-playground-series-sep-2021/test.csv", index_col=0)

About new features, i referred  https://www.kaggle.com/realtimshady/single-simple-lightgbm, thanks here.

In [ ]:
#create new features
features = [x for x in train.columns.values if x[0]=="f"]

train['n_missing'] = train[features].isna().sum(axis=1)
train['abs_sum'] = train[features].abs().sum(axis=1)
train['sem'] = train[features].sem(axis=1)
train['std'] = train[features].std(axis=1)
train['avg'] = train[features].mean(axis=1)
train['max'] = train[features].max(axis=1)
train['min'] = train[features].min(axis=1)

test['n_missing'] = test[features].isna().sum(axis=1)
test['abs_sum'] = test[features].abs().sum(axis=1)
test['sem'] = test[features].sem(axis=1)
test['std'] = test[features].std(axis=1)
test['avg'] = test[features].mean(axis=1)
test['max'] = test[features].max(axis=1)
test['min'] = test[features].min(axis=1)

For this experiment, I used HyperGBM, one kind of autoML tool.
Simple introduction, HyperGBM is a library that supports full-pipeline AutoML, which completely covers the end-to-end stages of data cleaning, preprocessing, feature generation and selection, model selection and hyperparameter optimization.It is a real-AutoML tool for tabular data. You can visit [HyperGBM](https://github.com/DataCanvasIO/HyperGBM) for more details.

##### For first step, there are some things i did:
1. custom search space.
2. search best models.
3. choice xgb models Top4, lgb models Top3, cat models Top2 (total 9 models) for next step.

*(run lightgbm model for once--cv5, i cost 150mins,and only 2 cpu is running, i donot know why.
so for this code, i only run xgboost ,it's so fast by using kaggle gpu)*

In [ ]:
## custom search space
ROOT = '/kaggle/input/d/wumingyang/resource/'
import sys
sys.path.append(ROOT)
from MySearchSpace import MyGeneralSearchSpaceGenerator

##### I searched the base models locally, so just need reload history file.

In [ ]:
from hypergbm import make_experiment
from hypernets.core.trial import TrialHistory
from hypernets.searchers import PlaybackSearcher

history_file = f"{ROOT}history_stepxgb.txt" ## get from my datasets.
target = 'claim'
reward = 'auc' 

search_space_ = MyGeneralSearchSpaceGenerator(class_balancing=None)
history = TrialHistory.load_history(search_space_, history_file)
## for search algorithm, hypergbm support evolution,mcts, random, but here, i just need reload history with PlaybackSearcher.
playback = PlaybackSearcher(history, top_n=9, optimize_direction='max')

##### About GreedyEnsemble:

    """
    References
    ----------
        Caruana, Rich, et al. "Ensemble selection from libraries of models." Proceedings of the twenty-first international conference on Machine learning. 2004.
    """

In [ ]:
##run make_experiment
exp = make_experiment(
                      train.copy(),
                      target=target,
                      reward_metric=reward,
                      cv = True,
                      num_folds= 5, ## 5 folds for Cross-validate
                      max_trials=4,
                      ensembel_size = 4, ## ensembel models with GreedyEnsembel
                      early_stopping_time_limit=3600*3, ##early stop for 3h
                      log_level = 'info',  ##some info will output while trainning
                      searcher = playback,
                      random_state=0)
estimator = exp.run()

In [ ]:
preds = estimator.predict_proba(test)[:,1]

In [ ]:
submit_df = pd.read_csv("../input/tabular-playground-series-sep-2021/sample_solution.csv")
submit_df['claim'] = preds
submit_df.to_csv("submission.csv", index=False)

In [ ]:
submit_df.head(3)

##### Try to do simple stacking.
1. get new_train,new_test from model's predict result.
2. train new_data

In [ ]:
# ## get new_train, new_test
# from hypergbm import HyperGBMEstimator

# new_train = np.zeros((train.shape[0],10)) ## 9 models's predict result + claim
# new_test = np.zeros((test.shape[0],9))

# for i,trial in enumerate(exp.hyper_model_.history.trials):  ## get oof result from exp
#     new_train[:,i] = trial.memo['oof'][:,1]  ##oof result saved in trial.memo for train data
    
#     model = HyperGBMEstimator.load(trial.model_file)  ##get trained model from model_file.
#     new_test[:,i] = model.predict_proba(test)[:,1]  
    
# new_train[:,-1] = train[target].values



In [ ]:
# # transform numpy to Dataframe
# _columns = [f'feature{i}' for i in range(9)]
# new_train = pd.DataFrame(new_train,columns=_columns+[target])
# new_test = pd.DataFrame(new_test,columns=_columns)
# new_train.head(3)

In [ ]:
# import catboost
# from sklearn.linear_model import LogisticRegression

# meta_model1 = LogisticRegression(max_iter=1000,multi_class='multinomial',solver='lbfgs')
# meta_model2 = catboost.CatBoostClassifier(depth=1,verbose=False)


In [ ]:
# iterators = StratifiedKFold(n_splits=5, shuffle=True,random_state=0)
# y = new_train.pop(target)
# X = new_train
# preds = np.zeros((test.shape[0],2)) ## for 2 models
# for n_fold, (train_idx, valid_idx) in enumerate(iterators.split(X, y)):
#     x_train_fold, y_train_fold = X.iloc[train_idx], y.iloc[train_idx]
#     x_val_fold, y_val_fold = X.iloc[valid_idx], y.iloc[valid_idx]
#     for i,meta_model in enumerate([meta_model1,meta_model2]):
#         meta_model.fit(x_train_fold,y_train_fold)
#         pred = meta_model.predict_proba(x_val_fold)[:,1]
#         preds[valid_idx,i] = pred
    
# ## just average the result
# preds = np.mean(preds,axis=1)

# # submission = pd.read_csv('../input/tabular-playground-series-sep-2021/sample_solution.csv', index_col='id')
# # submission['claim'] = preds
# # submission.to_csv('submission_simpleStacking.csv')

# ## PLB: 0.81825

##### Now, i have a new_train data which has 9 features,before move to the next step,add some new features.

In [ ]:

# from itertools import combinations
# def generate_new_features(df):
#         combs = list(combinations(df.columns, 2))
#         for i, j in combs:
#             column_name = '%s-%s' % (i, j)
#             df[column_name] = df[i] - df[j]
#             # column_name = '%s+%s' % (i, j)
#             # df[column_name] = df[i] + df[j]
#         return df

# new_train = generate_new_features(X)
# new_train[target] = y
# new_test = generate_new_features(new_test)